In [ ]:
#ADM_Assignment3_Team3

In [ ]:
#Indexing data to query Elasticsearch for similar images

In [ ]:
import json

# import time for its sleep method
from time import sleep

# import the datetime libraries datetime.now() method
from datetime import datetime

# use the elasticsearch client's helpers class for _bulk API
from elasticsearch import Elasticsearch, helpers

ModuleNotFoundError: ignored

In [ ]:
# declare a client instance of the Python Elasticsearch library
client = Elasticsearch("localhost:9200")

In [ ]:

# define a function that will load a text file
def get_data_from_text_file(self):
# the function will return a list of docs
    return [l.strip() for l in open(str(self), encoding="utf8", errors='ignore')]

# call the function to get the string data containing docs
docs = get_data_from_text_file("nearest_neighbours_json.json")

# print the length of the documents in the string
print ("String docs length:", len(docs))

String docs length: 132914


In [ ]:
import json
 
from tqdm import tqdm
 
from elasticsearch import Elasticsearch
 
es = Elasticsearch("localhost:9200")
 
def main():
    skus = json.load(open('/Users/harooniqbal/Desktop/new/NEU_SushmithaJogula/Semester3/ADM/Assignment3/nearest_neighbours_json.json'))
    payload = []
    for sku in tqdm(skus):
        payload.append(json.dumps({'index': {}}))
        payload.append(json.dumps(sku))
    es.bulk('\n'.join(payload), index='yg-vs-es-index', doc_type='_doc')
    es.indices.refresh(index='yg-vs-es-index')

if __name__ == '__main__':
    main()

100%|██████████| 5112/5112 [00:00<00:00, 57550.60it/s]
<ipython-input-11-56bedffc2093>:15: DeprecationWarning: Using positional arguments for APIs is deprecated and will be disabled in 8.0.0. Instead use only keyword arguments for all APIs. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.bulk('\n'.join(payload), index='yg-vs-es-index', doc_type='_doc')
/Users/harooniqbal/opt/anaconda3/lib/python3.8/site-packages/elasticsearch/connection/base.py:209: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)
/Users/harooniqbal/opt/anaconda3/lib/python3.8/site-packages/elasticsearch/connection/base.py:209: ElasticsearchWarning: [types removal] Specifying types in bulk requests is deprecated.
  war

In [ ]:
import base64
import json
import requests
import numpy as np
from tqdm import tqdm

# image_vec from image_src
def get_vector_from_source(image_src):
    tfs_endpoint = 'http://127.0.0.1/bd/v1/models/slim_inception_resnet_v2:predict'
  # image_str = requests.get(image_src).content
    image_str = open(image_src, 'rb').read()
    image_b64 = base64.b64encode(image_str).decode('utf-8')
    signature = dict(
        signature_name='omnium',
        instances=list([
            dict(
                    images=dict(
                    b64=image_b64
        )
      )
    ])
  )
    responses = requests.post(
    tfs_endpoint, json=signature
  )
    image_vec = responses.json().get('predictions', list([dict()]))[0].get('vector')
    return image_vec

# image_tkn from image_vec
def get_sign(x):
    if x >= 0:
        return '+'
    else:
        return '-'

def get_tokens_from_vector_sign(vector, n=256):
    assert vector is not None and n != 0, 'Invalid vector, or n.'
    tokens = [
        'i{index}v{value}'.format(
          index=i, value=get_sign(vector[i])
        )for i in np.argpartition(
      np.abs(vector), -n)[-n:]
  ]
    return tokens

def get_tokens_from_vector_round(vector, n=256, ndigits=0):
    assert vector is not None and n != 0 and ndigits >= -1, 'Invalid vector, n, or ndigits.'
    if ndigits == -1:
        return get_tokens_from_vector_sign(vector, n)
    tokens = [
    'i{index}v{value}'.format(
      index=i, value=round(vector[i], ndigits)
    )for i in np.argpartition(
      np.abs(vector), -n)[-n:]
  ]
    return tokens

def get_vecstr_from_vector(vector, precision=4):
    return ' '.join([str(round(v, precision)) for v in vector])

# main
def main():
    skus = json.load(open('./skus_insitu.json'))
    for sku in tqdm(skus):
        image_vec = get_vector_from_source(sku['image_src'].replace('http://127.0.0.1/images', './assets/insitu'))
        if image_vec is not None:
            sku['image_vec'] = get_vecstr_from_vector(image_vec)
            sku['image_tkn'] = get_tokens_from_vector_sign(image_vec)
        else:
            print('Invalid im2vec: {}'.format(sku['image_src']))
    with open('./skus_insitu_im2vec.json', 'w') as fp:
        json.dump(skus, fp)
    skus = json.load(open('./skus_onsite.json'))
    for sku in tqdm(skus):
        image_vec = get_vector_from_source(sku['image_src'].replace('http://127.0.0.1/images', './assets/onsite'))
        if image_vec is not None:
            sku['image_vec'] = get_vecstr_from_vector(image_vec)
            sku['image_tkn'] = get_tokens_from_vector_sign(image_vec)
        else:
            print('Invalid im2vec: {}'.format(sku['image_src']))
    with open('./skus_onsite_im2vec.json', 'w') as fp:
        json.dump(skus, fp)

if __name__ == '__main__':
    main()